# Reversible Network Reconnection Model

Attempt at implementing the RNR model as described in Okuda et al. 2012:

>_Reversible network reconnection model for simulating large deformation in dynamic tissue morphogenesis_, Satoru Okuda, Yasuhiro Inoue, Mototsugu Eiraku, Yoshiki Sasai and Taiji Adachi Biomech Model Mechanobiol (2013) 12:627–644 [DOI 10.1007/s10237-012-0430-7](http://link.springer.com/article/10.1007%2Fs10237-012-0430-7)


The authors give 4 conditions which we detail and implement bellow.




## Condition 1 - Center of a face

The center of a face is defined by the average position of the face's edges midpoints, weighted by their lengths.

$$ \mathbf{r}_{\alpha} = \frac{\sum_{ij\alpha}\ell_{ij} (\mathbf{r}_i + \mathbf{r}_j)/2}{\sum_{ij\alpha}\ell_{ij}}$$


In [1]:
import numpy as np, pandas as pd
import ipyvolume as ipv
import matplotlib.pyplot as plt
%matplotlib inline

from tyssue import Epithelium
from tyssue import BulkGeometry, RNRGeometry
from tyssue import Sheet
from tyssue.config.geometry import bulk_spec
from tyssue.generation import three_faces_sheet
from tyssue.generation import extrude
from tyssue.draw import sheet_view, highlight_cells

from tyssue.topology.bulk_topology import IH_transition, HI_transition



draw_spec = {'face': {'visible': True}}




sheet = Sheet.planar_sheet_3d('sheet', 5, 5, 1, 1)
sheet.sanitize()
datasets = extrude(sheet.datasets, method='translation')

eptm = Epithelium('20faces_3D', datasets, bulk_spec())
RNRGeometry.update_all(eptm)
RNRGeometry.center(eptm)
RNRGeometry.update_all(eptm)


eptm.settings['threshold_length'] = 1e-4
print(eptm.Nf, eptm.Ne, eptm.Nv)
draw_spec['face']['color'] = eptm.face_df.area
draw_spec['face']['color_range'] = (0, 1)

ipv.clear()
fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

93 414 70


Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…

## I→H transition

![IH transition in a bulk and a RNR](data/png/IH_transition.png)

This produces a shape similar to the one studied by LM Escudero and collegues [in their article](https://www.nature.com/articles/s41467-018-05376-1.pdf).





In [3]:
e_1011 = 26
eptm.settings['threshold_length'] = 0.5

srce, trgt, face, cell = eptm.edge_df.loc[e_1011,
                                          ['srce', 'trgt',
                                           'face', 'cell']]

IH_transition(eptm, e_1011)
RNRGeometry.update_all(eptm)

highlight_cells(eptm, cell, reset_visible=True)
draw_spec['face']['color'] = eptm.face_df.area


ipv.clear()
fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…

# H → I transition

In [5]:
eptm.Nv

71

In [8]:
eptm.active_verts.size

71

In [4]:
fa = 93


HI_transition(eptm, fa)
RNRGeometry.update_all(eptm)

highlight_cells(eptm, cell, reset_visible=False)
ipv.clear()
draw_spec['face']['color'] = eptm.face_df.area

fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

Figure(camera=PerspectiveCamera(fov=46.0, position=(0.0, 0.0, 2.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.…

### Testing for I→H / H →I transition triggers

In [4]:
#eptm.settings['threshold_length'] = 1e-3
eptm.settings['threshold_length'] = eptm.edge_df.length.min()+1e-3

In [5]:
def find_rearangements(eptm):
    l_th = eptm.settings['threshold_length']
    up_num_sides = eptm.upcast_face(eptm.face_df['num_sides'])   
    shorts = eptm.edge_df[eptm.edge_df['length'] < l_th]
    non_triangular = up_num_sides[up_num_sides > 4 ].index
    edges_IH = set(shorts.index).intersection(non_triangular)

    max_f_length = shorts.groupby('face')['length'].apply(max)
    short_faces = max_f_length[max_f_length < l_th].index
    three_faces = eptm.face_df[eptm.face_df['num_sides'] == 3].index
    faces_HI = set(three_faces).intersection(short_faces)
    return edges_IH, faces_HI

find_rearangements(eptm)

({410, 412}, set())

### Condition 3

This condition is satisfied if `eptm.settings['threshold_length']`
is well defined, i.e, small with respect to the unit length or the
average edge length.

### Condition 4

* (i)  Two edges never share two vertices simultaneously.

_In our half-edge architecture, this is not as straight forward. But I think the condition can
be reformulated as:_ Two edges **from the same face** never share two edges simultanously.
I don't know how to demonstrate this, but I think these are equivalent (to get two edges together, you need to "squeeze" one face between those two.

* (ii) Two polygonal faces never share two or more edges simultaneously.


In [6]:
from tyssue.topology.base_topology import condition_4i, condition_4ii

In [7]:
condition_4i(eptm), condition_4ii(eptm)

(Int64Index([], dtype='int64', name='face'), [])

In [9]:
eptm.settings['threshold_length'] = 0.5

IH_transition(eptm, eptm.edge_df.index[-1])
RNRGeometry.update_all(eptm)
draw_spec['face']['color'] = eptm.face_df.area

ipv.clear()
fig, meshes = sheet_view(eptm, mode='3D', **draw_spec)
fig

Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], meshes=[Mesh(color=array('#2b5d0a', dtype='<U7'), lines=array([[ 2,  0],
       [ 1,  2],
       [ 0,  4],
       [ 4,  3],
       [ 2,  5],
       [ 5,  2],
       [ 4, 71],
       [71,  4],
       [ 5, 68],
       [68,  5],
       [ 3,  6],
       [12, 11],
       [10, 13],
       [13, 12],
       [16, 14],
       [14, 15],
       [15, 18],
       [18, 15],
       [16, 69],
       [69, 16],
       [68, 17],
       [17, 68],
       [17, 18],
       [18, 17],
       [ 6, 16],
       [15, 10],
       [ 5,  8],
       [ 8,  5],
       [ 7,  8],
       [ 8,  7],
       [ 7, 17],
       [17,  7],
       [19,  1],
       [ 8, 20],
       [20,  8],
       [20, 19],
       [22,  9],
       [11, 24],
       [24, 22],
       [13, 25],
       [25, 13],
       [18, 23],
       [23, 18],
       [23, 25],
       [25, 23],
       [24, 25],
       [25, 24],
       [ 9, 26],
       [21, 20],
       [29, 27],
       [27, 28],
       [28, 30],
       [30, 28],
       [29, 32],
       [32, 29],
       [30, 31],
       [31, 30],
       [31, 32],
       [32, 31],
       [26, 29],
       [28, 21],
       [ 7, 30],
       [30,  7],
       [22, 32],
       [32, 22],
       [23, 31],
       [31, 23],
       [33, 35],
       [35, 34],
       [37, 33],
       [36, 37],
       [38, 35],
       [35, 38],
       [39, 37],
       [37, 39],
       [39, 38],
       [38, 39],
       [40, 36],
       [45, 46],
       [47, 44],
       [46, 47],
       [48, 50],
       [49, 48],
       [53, 49],
       [49, 53],
       [51, 50],
       [50, 51],
       [52, 51],
       [51, 52],
       [53, 52],
       [52, 53],
       [50, 40],
       [44, 49],
       [51, 39],
       [39, 51],
       [42, 38],
       [38, 42],
       [42, 41],
       [41, 42],
       [52, 41],
       [41, 52],
       [34, 54],
       [55, 42],
       [42, 55],
       [54, 55],
       [43, 57],
       [59, 45],
       [57, 59],
       [60, 47],
       [47, 60],
       [58, 53],
       [53, 58],
       [60, 58],
       [58, 60],
       [60, 59],
       [59, 60],
       [61, 43],
       [55, 56],
       [62, 64],
       [63, 62],
       [65, 63],
       [63, 65],
       [67, 64],
       [64, 67],
       [66, 65],
       [65, 66],
       [67, 66],
       [66, 67],
       [64, 61],
       [56, 63],
       [65, 41],
       [41, 65],
       [67, 57],
       [57, 67],
       [66, 58],
       [58, 66],
       [ 0,  2],
       [ 2, 35],
       [35, 33],
       [33,  0],
       [ 2,  1],
       [ 1, 34],
       [34, 35],
       [35,  2],
       [ 4,  0],
       [ 0, 33],
       [33, 37],
       [37,  4],
       [ 3,  4],
       [ 4, 37],
       [37, 36],
       [36,  3],
       [ 5,  2],
       [ 2, 35],
       [35, 38],
       [38,  5],
       [ 2,  5],
       [ 5, 38],
       [38, 35],
       [35,  2],
       [71,  4],
       [ 4, 37],
       [37, 39],
       [39, 71],
       [ 4, 71],
       [71, 39],
       [39, 37],
       [37,  4],
       [68,  5],
       [ 5, 38],
       [38, 39],
       [39, 71],
       [ 5, 68],
       [71, 39],
       [39, 38],
       [38,  5],
       [ 6,  3],
       [ 3, 36],
       [36, 40],
       [40,  6],
       [11, 12],
       [12, 46],
       [46, 45],
       [45, 11],
       [13, 10],
       [10, 44],
       [44, 47],
       [47, 13],
       [12, 13],
       [13, 47],
       [47, 46],
       [46, 12],
       [14, 16],
       [16, 50],
       [50, 48],
       [48, 14],
       [15, 14],
       [14, 48],
       [48, 49],
       [49, 15],
       [18, 15],
       [15, 49],
       [49, 53],
       [53, 18],
       [15, 18],
       [18, 53],
       [53, 49],
       [49, 15],
       [69, 16],
       [16, 50],
       [50, 51],
       [51, 69],
       [16, 69],
       [69, 51],
       [51, 50],
       [50, 16],
       [17, 68],
       [69

In [15]:
eptm.validate()

False

In [16]:
bad_edges = eptm.edge_df[~eptm.edge_df['is_valid']].index

In [17]:
bad_edges

Int64Index([  0,   2,   3,   5,   6,   7,   9,  10,  14,  15,
            ...
            411, 412, 413, 417, 419, 421, 425, 426, 427, 428],
           dtype='int64', name='edge', length=111)

In [7]:
invalid = eptm.get_invalid()
assert np.alltrue(1 - invalid)
assert np.alltrue(eptm.edge_df['sub_vol'] > 0)

AssertionError: 

In [10]:
%pdb

Automatic pdb calling has been turned ON
